In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist
import pandas as pd
import os

In [26]:
# HYPERPARAMETERS
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001

train_df = pd.read_csv("Multi_Digit_Mnist/train.csv")
test_df = pd.read_csv("Multi_Digit_Mnist/test.csv")
train_images = os.getcwd() + "/Multi_Digit_Mnist/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/Multi_Digit_Mnist/test_images/" + test_df.iloc[:, 0].values

In [27]:
train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values


def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)


In [28]:
inputs = keras.Input(shape=(64,64,1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding='same',
    kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
)(inputs)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(64, 3, kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_regularizer=regularizers.l2(WEIGHT_DECAY))(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
output1 = layers.Dense(10, activation='softmax',name='first_num')(x)
out_put2 = layers.Dense(10, activation='softmax',name='second_num')(x)

In [29]:
model = keras.Model(inputs=inputs, outputs=[output1,out_put2])

In [30]:
model.compile(
    optimizer=keras.optimizers.Adam(LEARNING_RATE),
    loss=[
        keras.losses.SparseCategoricalCrossentropy(),
        keras.losses.SparseCategoricalCrossentropy(),
    ],
    metrics=["accuracy"],
)

In [31]:
model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

Epoch 1/5
1000/1000 - 675s - loss: 2.0820 - first_num_loss: 1.0000 - second_num_loss: 1.0059 - first_num_accuracy: 0.6410 - second_num_accuracy: 0.6352 - 675s/epoch - 675ms/step
Epoch 2/5
1000/1000 - 410s - loss: 0.6663 - first_num_loss: 0.3030 - second_num_loss: 0.3055 - first_num_accuracy: 0.8992 - second_num_accuracy: 0.8967 - 410s/epoch - 410ms/step
Epoch 3/5
1000/1000 - 443s - loss: 0.4358 - first_num_loss: 0.1922 - second_num_loss: 0.1907 - first_num_accuracy: 0.9373 - second_num_accuracy: 0.9375 - 443s/epoch - 443ms/step
Epoch 4/5
1000/1000 - 525s - loss: 0.3363 - first_num_loss: 0.1462 - second_num_loss: 0.1399 - first_num_accuracy: 0.9522 - second_num_accuracy: 0.9543 - 525s/epoch - 525ms/step
Epoch 5/5
1000/1000 - 506s - loss: 0.2753 - first_num_loss: 0.1147 - second_num_loss: 0.1118 - first_num_accuracy: 0.9629 - second_num_accuracy: 0.9626 - 506s/epoch - 506ms/step
313/313 - 104s - loss: 1.2462 - first_num_loss: 0.4062 - second_num_loss: 0.7928 - first_num_accuracy: 0.8795 

[1.246191143989563,
 0.40618160367012024,
 0.792787492275238,
 0.8794500231742859,
 0.7984499931335449]